In [ ]:
import os
from pathlib import Path

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In this tutorial we will use the same building as the one describe in the Building tutorial. Please refer to this file to understand <code>Building</code>.
The use case is a 2 floors 4 apartments residential building.
# Energytool Systems and Modifiers

This notebook demonstrates how to define and apply Energy Efficiency Measures (EEMs) and building systems using `energytool` and `corrai`.

We distinguish two complementary layers of building model customization:

- **Systems**: These include heating, cooling, ventilation, lighting, domestic hot water (DHW), sensors, etc. They are added via `building.add_system(...)` and simulate physical systems or monitoring logic. Each system inherits from the `System` base class and must define how it affects the simulation, before and/or after it runs.

- **Modifiers**: These directly modify the IDF (EnergyPlus input file) to apply specific design or operational changes (window types, wall composition, airflow settings, etc.). They are applied via dedicated functions such as `set_external_windows`, `set_opaque_surface_construction`, and others in the `energytool.modifier` module.

In this tutorial, we explore how to:
- Manually assign building systems to simulate realistic energy usage (e.g. boiler, heat pump, ventilation).
- Define modifiers and apply them to the IDF to implement EEMs.
- Combine multiple variants and simulate all combinations using `simulate_variants()` from the `corrai` library.
- Treat both system types (like `HeaterSimple`, `AirHandlingUnit`) and modifiers as unified elements in the variant dictionary, allowing easy scenario generation and comparison.

This approach enables a flexible way to evaluate the energy performance of various design and retrofit strategies using Python.

## Building systems

Building systems represent the core technical components that define how the building operates: heating, cooling, ventilation, lighting, hot water, renewable production, sensors, and more. These systems are added programmatically to the `Building` object using the `add_system()` method and are defined as subclasses of the abstract `System` base class.

Each system defines its own logic through two key methods:
- `pre_process(idf)`: to prepare or modify the IDF before simulation (e.g., adding output variables).
- `post_process(idf, results)`: to process simulation results and calculate performance indicators (e.g., energy use, thermal comfort).

This modular structure makes it easy to plug in custom systems (like `HeaterSimple`, `AirHandlingUnit`, or `Overshoot28`) and simulate their impact in a reusable and traceable way.
Below are a few examples of possible systems.

In [ ]:
from energytool.building import Building, SimuOpt
Building.set_idd(Path(r"C:\EnergyPlusV9-4-0"))

from energytool.system import (
    HeaterSimple,
    HeatingAuxiliary,
    AirHandlingUnit,
    AHUControl,
    DHWIdealExternal,
    ArtificialLighting,
    Overshoot28,
    Sensor
)

In [ ]:
building = Building(idf_path=Path(TUTORIAL_DIR) / "resources/tuto_building.idf")

In [ ]:
zones = [
    "Block1:ApptX1W",
    "Block1:ApptX1E1",
    "Block1:ApptX1E",
    "Block2:ApptX2W",
    "Block2:ApptX2E",
]

In [ ]:
# Simulate a boiler, multiplying the heat needs by a constant COP
building.add_system(HeaterSimple(
    name="Gaz_boiler", 
    zones = zones,
    cop=0.89
))


# Estimate circulation pumps energy consumption multiplying the heat needs by a constant (default 0.05)
building.add_system(HeatingAuxiliary(
    name="Circulation_pumps", 
    zones = zones,
    ratio=0.05,
))


# Simulate fan consumption multiplying extracted air volume by a constant coefficient
# Do not have a heat exchanger
building.add_system(AirHandlingUnit(
    name="Extraction_fan", 
    zones = zones,
    fan_energy_coefficient=0.23, # Wh/m3
    heat_recovery_efficiency=False
))


# Simulate clock regulation
# Ventilation works according to specified schedule
#  is defined in the energytool/resources/resources_idf.idf file
building.add_system(AHUControl(
    name="AHU_control", 
    zones = zones,
    control_strategy="Schedule",
    schedule_name="OFF_09h_18h_ON_18h_24h_ON_WE_FULL_YEAR",
))


# Estimate Domestic Hot Water production energy needs
# Use the number of people defined in the idf file to estimate the total volume.
# Otherwise, energy calculation is independent of energyplus
building.add_system(DHWIdealExternal(
    name="DHW_Electric_accumulation", 
    zones = zones,
    cop=0.85
))


# Estimate Lighting consumption using a constant power ratio.
# Modify the existing energyplus object
building.add_system(ArtificialLighting(
    name="Random_lights",
    zones = zones,
    power_ratio=4 # W/m²
))
        
# Add variables for summer thermal comfort calculation
building.add_system(Sensor(
    name="ZOP", 
    variables=["Zone Operative Temperature"],
    key_values="*"
))

building.add_system(Overshoot28(
    name="thermal comfort",
    temp_threshold=27,
    occupancy_in_output=True
))

In [ ]:
building

## Building modifiers

<b>Define and Apply Variants using energytool.modifier and corrai.variant</b>

Building modifiers are functions that directly alter the geometry, construction, or control logic of the EnergyPlus model (IDF). These modifiers allow you to apply Energy Efficiency Measures (EEMs) by programmatically changing window properties, wall compositions, airflow openings, blinds, and more.

Modifiers are implemented as standalone functions in the `energytool.modifier` module. Each modifier takes the `Building` model and a description dictionary as input, and updates the IDF accordingly. For example, `set_external_windows()` can change the glazing type of selected surfaces, while `set_opaque_surface_construction()` can modify the wall layers.

These modifiers can be called manually or integrated into a variant workflow, allowing you to define a set of design alternatives and automatically simulate their impact.

In [ ]:
from energytool.modifier import (
    set_external_windows, 
    set_opaque_surface_construction,
    set_system
)

For instance, let's try to modify the insulation composition. 
It must be defined in a dictionary, then used in function <code>set_opaque_surface_construction

In [ ]:
wall_insulation_compo = {
    "EEM1_Wall_int_insulation" : [
        {
            "Name": "Project medium concrete block_.2",
            "Thickness": 0.2,
            "Conductivity": 0.51,
            "Density": 1400,
            "Specific_Heat": 1000,
        },
        {
            "Name": "Laine_15cm",
            "Thickness": 0.15,
            "Conductivity": 0.032,
            "Density": 40,
            "Specific_Heat": 1000,
        },
    ]
}

set_opaque_surface_construction(
    model=building,
    surface_type="Wall",
    outside_boundary_condition="Outdoors",
    description=wall_insulation_compo
)  

Let's check if materials with names "Laine_15cm" and "Project medium concrete block_.2" do exist with the correct defined properties, as well as the construction "EEM1_Wall_int_insulation", by looking into the idf.

In [ ]:
for material in building.idf.idfobjects["MATERIAL"]:
    if "Laine_15cm" in material.Name or "Project medium concrete block_.2" in material.Name:
        print(material)

In [ ]:
for construction in building.idf.idfobjects["CONSTRUCTION"]:
    if "EEM1_Wall_int_insulation" in construction.Name:
        print(construction)

All good. 

Likewise, we can change various properties/composition using the other functions:  
- <code>set_external_windows</code>: Replace windows in an EnergyPlus building model with new window descriptions: UFactor, Solar_Heat_Gain_Coefficient, Visible_Transmittance
- <code>set_afn_surface_opening_factor</code>: Modify AirFlowNetwork:Multizone:Surface WindowDoor_Opening_Factor_or_Crack_Factor
    based on their name
- <code>set_blinds_solar_transmittance</code> : Modify WindowMaterial:Shade Solar_Transmittance (or/and Reflectance) based
    on the given description.

Here is an example for window modifications:

In [ ]:
window_modifier = {
    "Window_variant": {
        "Name": "Simple window_B4R_simple - modified",
        "UFactor": 1.3,
        "Solar_Heat_Gain_Coefficient": 0.5,
        "Visible_Transmittance": 0.8
    }
}

set_external_windows(
    model=building,
    description=window_modifier,
    boundary_conditions = "Outdoors"
)

In [ ]:
building.idf.idfobjects["WindowMaterial:SimpleGlazingSystem"]

When evaluating multiple design or system variants, manually applying and simulating each one can quickly become tedious and error-prone.

To streamline this process, we use the `corrai` library, which offers a structured and automated workflow for:

- Defining multiple variants (modifiers or systems),
- Generating combinations of those variants (e.g., renovation packages),
- Applying the corresponding changes to the building model,
- Launching simulations for all selected combinations in parallel if needed.

This approach enables rapid testing of Energy Efficiency Measures (EEMs), system alternatives, or control strategies in a reproducible and scalable way.

The simulation pipeline is organized into a few simple steps, described below.

### Step 1 - Write variants and associated modifiers

The variants for windows, walls, blinds, apertures, systems, etc., are explained in a dictionary, using enum from <code>VriantKeys</code> of library **corrAI**.

In [ ]:
from corrai.variant import VariantKeys

In [ ]:
variant_dict = {
    "EEM_Wall_int_insulation": {
        VariantKeys.MODIFIER: "walls_modifier", 
        VariantKeys.ARGUMENTS: {
            "name_filter": "", # all, but could be "ApptX1W" for instance
            "surface_type": "Wall"
        },
        VariantKeys.DESCRIPTION: {
            "EEM1_Wall_int_insulation": [
                {
                    "Name": "Project medium concrete block_.2",
                    "Thickness": 0.2,
                    "Conductivity": 0.51,
                    "Density": 1400,
                    "Specific_Heat": 1000,
                },
                {
                    "Name": "Laine_15cm",
                    "Thickness": 0.15,
                    "Conductivity": 0.032,
                    "Density": 40,
                    "Specific_Heat": 1000,
                },
            ]
        }
    },
    "EEM_Wall_ext_insulation": {
        VariantKeys.MODIFIER: "walls_modifier",
        VariantKeys.ARGUMENTS: {
            "name_filter": "",
            "surface_type": "Wall"
        },
        VariantKeys.DESCRIPTION: {
            "EEM2_Wall_ext_insulation": [
                {
                    "Name": "Coating",
                    "Thickness": 0.01,
                    "Conductivity": 0.1,
                    "Density": 400,
                    "Specific_Heat": 1200,
                },
                {
                    "Name": "Laine_30cm",
                    "Thickness": 0.30,
                    "Conductivity": 0.032,
                    "Density": 40,
                    "Specific_Heat": 1000,
                },
                {
                    "Name": "Project medium concrete block_.2",
                    "Thickness": 0.2,
                    "Conductivity": 0.51,
                    "Density": 1400,
                    "Specific_Heat": 1000,
                },
            ]
        }
    },
    "EEM_Double_glazing": {
        VariantKeys.MODIFIER: "windows",
        VariantKeys.ARGUMENTS: {
            "surface_name_filter": ""
        },
        VariantKeys.DESCRIPTION: {
            "EEM3_Double_glazing": {
                "Name": "Double_glazing",
                "UFactor": 1.1,
                "Solar_Heat_Gain_Coefficient": 0.41,
                "Visible_Transmittance": 0.71,
            }
        }
    },

    "EEM_simple_glazing": {
        VariantKeys.MODIFIER: "windows",
        VariantKeys.ARGUMENTS: {
            "surface_name_filter": ""
        },
        VariantKeys.DESCRIPTION: {
            "EEM3_Single_glazing": {
                "Name": "Single_glazing",
                "UFactor":2,
                "Solar_Heat_Gain_Coefficient": 0.4,
                "Visible_Transmittance": 0.81,
            }
        }
    },

    "EEM_Gaz_Boiler": {
        VariantKeys.MODIFIER: "heating_system",
        VariantKeys.ARGUMENTS: {
            "system_name": "Main_boiler"
        },
        VariantKeys.DESCRIPTION: {
            "EEM1_Gaz_Boiler": HeaterSimple(
                name="Gaz_boiler",
                zones=zones,
                cop=0.89
            )
        }
    },
    "EEM_Heat_Pump": {
        VariantKeys.MODIFIER: "heating_system",
        VariantKeys.ARGUMENTS: {
            "system_name": "Main_boiler"
        },
        VariantKeys.DESCRIPTION: {
            "EEM2_Heat_Pump": HeaterSimple(
                name="PAC",
                zones=zones,
                cop=3.0
            )
        }
    }
}

### Step 2 - Define the Modifier Map

Generate a dictionary that maps modifier values (name) to associated variant names and list of combined variants based on the provided variant dictionary.

Each key of the <code>MOD_MAP</code> dictionary corresponds to a variant name (as found in the variant dictionnary descriptions), and each value is a function from <code>energytool.modifier</code> that knows how to apply that variant to the model.

In [ ]:
from corrai.variant import (
    VariantKeys,
    simulate_variants,
    get_combined_variants,
    get_modifier_dict,
)

In [ ]:
mod_map = {
    "walls_modifier": set_opaque_surface_construction,
    "windows": set_external_windows,
    "heating_system": set_system,
}

In [ ]:
mod_dict = get_modifier_dict(variant_dict)

In [ ]:
mod_dict

Here are all the possible combinations, using <code>get_combined_variants</code>.

In [ ]:
combinations = get_combined_variants(variant_dict)

In [ ]:
combinations

### Step 3 - Run simulations

Let's now run all simulations and use <code>simulate_variants</code> from **CorrAI**.

Optional arguments are:
- <code>n_cpu</code> for the number of CPU cores to use for parallel execution. Default is -1  meaning all CPUs but one, 0 is all CPU, 1 is sequential, >1 is the numbe   of cp
- <code>save_path</code> if you want to save each generated idf files. You should specify the file extension "idf" in file_extension.
- <code>custom_combination</code> if you pre-filtered combinations from get_combined_variants applied on VARIANT_DICT
- <code>add_existing</code> (default to False) if you want to add to your combinations existing scenarios not already specified in the variant dictionary.

In [ ]:
sim_opt = {
    SimuOpt.EPW_FILE.value: Path(TUTORIAL_DIR) / r"resources/FRA_Bordeaux.075100_IWEC.epw",
    SimuOpt.OUTPUTS.value: "SYSTEM|SENSOR",
    SimuOpt.START.value: "2025-01-01",
    SimuOpt.STOP.value: "2025-01-19",
    SimuOpt.VERBOSE.value: "v", 
}

In [ ]:
result = simulate_variants(
    model=building,
    variant_dict=variant_dict,
    modifier_map=mod_map,
    simulation_options=sim_opt,
    n_cpu=1,
)

### Step 4 - Compare results

In [ ]:
joules_to_kWhef = 1 / 3.6e6

for i, res in enumerate(result):
    result[i]["TOTAL_SYSTEM_Energy_(kWh)"] = res["TOTAL_SYSTEM_Energy_[J]"] * joules_to_kWhef
    temp_cols = [col for col in res.columns if "Zone Operative Temperature" in col]
    result[i]["Zone Operative Temperature mean_(°C)"] = res[temp_cols].mean(axis=1)

In [ ]:
import plotly.express as px
import pandas as pd

all_results = pd.concat(
    [
        res.assign(Scenario=str(combinations[i]))
        for i, res in enumerate(result)
    ],
    ignore_index=False 
)

fig = px.line(
    all_results,
    x=all_results.index,
    y="TOTAL_SYSTEM_Energy_(kWh)",
    color="Scenario",
    title="Total System Energy (kWh) over time for all scenarios"
)

fig.update_layout(
    legend_title_text="Scenario",
    legend_orientation="h",
    legend_yanchor="bottom",
    legend_y=-1.5,
    legend_xanchor="center",
    legend_x=0.5
)
fig.show()

In [ ]:
import pandas as pd

summary = pd.DataFrame([
    {
        "Mean Zone Operative Temperature (°C)": res["Zone Operative Temperature mean_(°C)"].mean(),
        "Total System Energy (kWh)": res["TOTAL_SYSTEM_Energy_(kWh)"].sum()
    }
    for res in result
], index=combinations)

summary